In [1]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor, AgentType, initialize_agent, load_tools
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import (
    JSONAgentOutputParser,
    ReActSingleInputOutputParser,
)
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.tools.render import render_text_description_and_args

from llamp.mp.tools import (
    MaterialsBonds,
    MaterialsDielectric,
    MaterialsElasticity,
    MaterialsMagnetism,
    MaterialsOxidation,
    MaterialsPiezoelectric,
    MaterialsRobocrystallographer,
    MaterialsSimilarity,
    MaterialsStructure,
    MaterialsSummary,
    MaterialsSynthesis,
    MaterialsTasks,
    MaterialsThermo,
)

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)

In [2]:
llm = ChatOpenAI(
    temperature=0, 
    # model='gpt-3.5-turbo-16k-0613',
    model='gpu-4-1106-preview',
    openai_api_key=OPENAI_API_KEY
)

tools = load_tools(["llm-math"], llm=llm)
tools += [
    MaterialsSummary(),
    MaterialsSynthesis(),
    MaterialsThermo(),
    MaterialsElasticity(),
    MaterialsMagnetism(),
    MaterialsDielectric(),
    MaterialsPiezoelectric(),
    MaterialsRobocrystallographer(),
    MaterialsOxidation(),
    MaterialsBonds(),
    MaterialsSimilarity(),
]

prompt = hub.pull("hwchase17/react-multi-input-json")
prompt = prompt.partial(
    tools=render_text_description_and_args(tools),
    tool_names=", ".join([t.name for t in tools]),
)

print(prompt)

llm_with_stop = llm.bind(stop=["\nObservation"])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_stop
    | JSONAgentOutputParser()
    # add a summarizer to take care of large outputs
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke(
    {
        "input": "What is the most stiff materials with lowest formation energy in Si-O system? You may use different tools to get the most holistic answer."
    }
)

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:227: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


input_variables=['agent_scratchpad', 'input'] partial_variables={'tools': 'Calculator: Useful for when you need to answer questions about math., args: {\'tool_input\': {\'type\': \'string\'}}\nsearch_materials_summary__get: u, args: {\'material_ids\': {\'anyOf\': [{\'type\': \'string\'}, {\'type\': \'null\'}], \'default\': None, \'description\': \'Comma-separated list of material_ids to query on\', \'title\': \'Material Ids\'}, \'formula\': {\'anyOf\': [{\'type\': \'string\'}, {\'type\': \'null\'}], \'default\': None, \'description\': \'Query by formula including anonymized formula or by including wild cards. A comma delimited string list of anonymous formulas or regular formulas can also be provided.\', \'title\': \'Formula\'}, \'chemsys\': {\'anyOf\': [{\'type\': \'string\'}, {\'type\': \'null\'}], \'default\': None, \'description\': \'A comma delimited string list of chemical systems. Wildcards for unknown elements only supported for single chemsys queries\', \'title\': \'Chemsys\'}

/home/cyrus/miniconda3/envs/llamp/lib/python3.11/site-packages/mp_api/client/mprester.py:227: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(


KeyboardInterrupt: 